# GabeDA Features (Product-Daily Performance)

This notebook creates product-daily performance metrics by aggregating transactions.
It analyzes each product's performance on each day of the month.

**Input:** Preprocessed transactions from 01_transactions notebook  
**Output:** Product-daily metrics (1 row per product per day)  
**Group By:** `product_id`, `dt_date`

## 1. Setup: Imports, Context Loading, Logging

## 0. Project Root Setup (Auto-generated)

In [1]:
# Auto-detect project root and add to Python path
import os
import sys
from pathlib import Path

# Get the project root (2 levels up from notebooks/development or notebooks/from_store)
notebook_dir = Path.cwd() if '__file__' not in globals() else Path(__file__).parent
project_root = notebook_dir.parent.parent

# Change to project root
os.chdir(project_root)

# Add project root to Python path if not already there
if str(project_root) not in sys.path:
    sys.path.insert(0, str(project_root))

print(f"Working directory: {os.getcwd()}")
print(f"Project root: {project_root}")

Working directory: c:\Projects\play\khujta_ai_business
Project root: c:\Projects\play\khujta_ai_business


In [2]:
import pandas as pd
import numpy as np

# v2.0 Refactored imports
from src.utils.logger import setup_logging, get_logger
from src.core.context import GabedaContext
from src.core.persistence import load_context_state, get_latest_state, save_context_state
from src.core.constants import *
from src.features.store import FeatureStore
from src.features.resolver import DependencyResolver
from src.features.detector import FeatureTypeDetector
from src.features.analyzer import FeatureAnalyzer
from src.execution.calculator import FeatureCalculator
from src.execution.groupby import GroupByProcessor
from src.execution.executor import ModelExecutor
from src.export.excel import ExcelExporter

# Load latest context state
client_name = 'test_client'
latest_state = get_latest_state(client_name, base_dir='data/context_states')

if latest_state:
    ctx, base_cfg = load_context_state(latest_state)
    print(f"✓ Loaded latest state: {latest_state}")
else:
    raise FileNotFoundError(f"No context state found for client '{client_name}'")

# Setup logging
setup_logging(log_level=base_cfg.get('log_level', 'INFO'), 
              config={'client': base_cfg.get('client', 'unknown_client')})
logger = get_logger(__name__)

print(f"\n✓ Context loaded successfully!")
print(f"  - Original run_id: {ctx.original_run_id}")
print(f"  - New run_id: {ctx.run_id}")
print(f"  - Available datasets: {len(ctx.list_datasets())} datasets")

✓ Loaded latest state: data\context_states\test_client_20251022_150907
📝 Run instance ID: test_client_20251022_151002 - Logging [INFO] to: logs\test_client_20251022_151002.log

✓ Context loaded successfully!
  - Original run_id: test_client_20251022_150949
  - New run_id: test_client_20251022_151001
  - Available datasets: 7 datasets


## 2. Load Input Data

In [3]:
# Get input dataset
input_df = ctx.get_dataset('transactions_filters')

print(f"✓ Input dataset loaded")
print(f"  - Shape: {input_df.shape}")
print(f"  - Date range: {input_df['dt_date'].min()} to {input_df['dt_date'].max()}")
print(f"  - Unique products: {input_df['in_product_id'].nunique()}")
print(f"\nFirst few rows:")
input_df.head()

✓ Input dataset loaded
  - Shape: (609, 59)
  - Date range: 20251001 to 20251030
  - Unique products: 10

First few rows:


,in_dt,in_product_id,in_quantity,in_price_total,in_trans_type,in_customer_id,in_description,in_category,in_unit_type,in_stock,...,cost_unit,cost_total,price_unit,price_total,margin_unit,margin_unit_pct,margin_unit_valid,margin_total,margin_total_pct,margin_total_valid
0,2025-10-01 01:02:00,prod8,2.0,52964.0,return,client13,product 8,category B,pack,61.0,...,18792.0,37585.0,26482.0,52964.0,7690.0,29.04,True,15379.0,29.04,True
1,2025-10-01 06:24:00,prod4,6.0,177195.0,sale,client6,product 4,category B,unit,30.0,...,21526.0,129155.0,29533.0,177195.0,8007.0,27.11,True,48040.0,27.11,True
2,2025-10-01 08:38:00,prod7,2.0,70492.0,return,client12,product 7,category A,unit,78.0,...,25754.0,51509.0,35246.0,70492.0,9492.0,26.93,True,18983.0,26.93,True
3,2025-10-01 09:59:00,prod2,4.0,86751.0,sale,client3,product 2,category A,unit,80.0,...,12947.0,51786.0,21688.0,86751.0,8741.0,40.30,True,34965.0,40.31,True
4,2025-10-01 10:07:00,prod3,3.0,76465.0,sale,client12,product 3,category B,unit,47.0,...,16943.0,5083.0,25488.0,76465.0,8545.0,33.53,True,71382.0,93.35,True


## 3. Define Features

Product-daily performance metrics:  
- Customer engagement (unique customers, transactions)
- Sales volume (quantity sum, mean, median)
- Revenue metrics (total sales, total cost, average transaction value)

In [4]:
# ===== Product-Daily Performance Metrics =====
# Aggregates transactions by product and date

# --- Customer Engagement ---
def customer_id_count(customer_id):
    """
    Count unique customers who purchased this product on this day.
    Formula: COUNT(DISTINCT customer_id)
    """
    return len(np.unique(customer_id))

def trans_id_count(trans_id):
    """
    Count unique transactions for this product on this day.
    Formula: COUNT(DISTINCT trans_id)
    """
    return len(np.unique(trans_id))

# --- Sales Volume ---
def quantity_sum(quantity):
    """
    Total quantity sold for this product on this day.
    Formula: SUM(quantity)
    """
    return np.sum(quantity)

def quantity_mean(quantity):
    """
    Average quantity per transaction for this product on this day.
    Formula: AVG(quantity)
    """
    return round(np.mean(quantity), 2)

def quantity_median(quantity):
    """
    Median quantity per transaction for this product on this day.
    Formula: MEDIAN(quantity)
    """
    return round(np.median(quantity), 2)

# --- Revenue Metrics ---
def price_total_sum(price_total):
    """
    Total revenue for this product on this day.
    Formula: SUM(price_total)
    """
    return np.sum(price_total)

def cost_total_sum(cost_total):
    """
    Total cost for this product on this day.
    Formula: SUM(cost_total)
    """
    return np.sum(cost_total)

def transaction_mean(price_total_sum, trans_id_count):
    """
    Average revenue per transaction for this product on this day.
    Formula: price_total_sum / trans_id_count
    """
    if trans_id_count == 0:
        return DEFAULT_FLOAT
    return round(price_total_sum / trans_id_count, 2)

print("✓ Feature functions defined: 8 attributes")

✓ Feature functions defined: 8 attributes


## 4. Configure Model

In [5]:
# Collect features into dictionary
features = {
    'customer_id_count': customer_id_count,
    'trans_id_count': trans_id_count,
    'quantity_sum': quantity_sum,
    'quantity_mean': quantity_mean,
    'quantity_median': quantity_median,
    'price_total_sum': price_total_sum,
    'cost_total_sum': cost_total_sum,
    'transaction_mean': transaction_mean,
}

# Model configuration
cfg_model = {
    'model_name': 'product_daily',
    'input_dataset_name': 'transactions_filters',
    'group_by': ['in_product_id', 'dt_date'],  # Product-day granularity
    'row_id': 'in_trans_id',
    'output_cols': list(features.keys()),
    'features': features,
}

print(f"✓ Model configured: '{cfg_model['model_name']}'")
print(f"  - Group by: {cfg_model['group_by']}")
print(f"  - Output features: {len(cfg_model['output_cols'])}")

✓ Model configured: 'product_daily'
  - Group by: ['in_product_id', 'dt_date']
  - Output features: 8


## 5. Prepare Features (Store, Resolve Dependencies, Save Config)

In [6]:
# Initialize feature store and store features
feature_store = FeatureStore()
feature_store.store_features(features, model_name=cfg_model['model_name'], auto_save=True)

# Resolve dependencies
resolver = DependencyResolver(feature_store)
in_cols, exec_seq, ext_cols = resolver.resolve_dependencies(
    output_cols=cfg_model['output_cols'],
    available_cols=input_df.columns.tolist(),
    group_by=cfg_model.get('group_by'),
    model=cfg_model['model_name']
)

# Update model config with resolved dependencies
cfg_model['in_cols'] = in_cols
cfg_model['exec_seq'] = exec_seq
cfg_model['ext_cols'] = ext_cols

# Save master configuration
feature_store.save_master_config(
    model_name=cfg_model['model_name'],
    model_config=cfg_model
)

print("✓ Features prepared and dependencies resolved")
print(f"  - Input columns needed: {len(in_cols)}")
print(f"  - Execution sequence: {exec_seq}")
print(f"  - Master config saved: feature_store/{cfg_model['model_name']}/master_cfg.json")

✓ Features prepared and dependencies resolved
  - Input columns needed: 5
  - Execution sequence: ['customer_id_count', 'trans_id_count', 'quantity_sum', 'quantity_mean', 'quantity_median', 'price_total_sum', 'cost_total_sum', 'transaction_mean']
  - Master config saved: feature_store/product_daily/master_cfg.json


## 6. Execute Model (Initialize Components + Execute + Store Results)

In [7]:
# Initialize execution components
detector = FeatureTypeDetector()
analyzer = FeatureAnalyzer(feature_store, detector)
calculator = FeatureCalculator()
groupby_processor = GroupByProcessor(calculator, detector)
executor = ModelExecutor(analyzer, groupby_processor, context=ctx)

# Execute model
output = executor.execute_model(
    cfg_model=cfg_model,
    input_dataset_name=cfg_model['input_dataset_name']
)

# Store results in context
ctx.set_model_output(cfg_model['model_name'], output, cfg_model)

print("✓ Model executed successfully!")
print(f"  - Filters: {output['filters'].shape if output['filters'] is not None else 'None'}")
print(f"  - Attributes: {output['attrs'].shape if output['attrs'] is not None else 'None'}")
print(f"  - Product-day combinations: {output['attrs'].shape[0] if output['attrs'] is not None else 0}")

✓ Model executed successfully!
  - Filters: (609, 59)
  - Attributes: (266, 10)
  - Product-day combinations: 266


## 7. View Results

In [8]:
# View product-daily performance (aggregated attributes)
attrs = ctx.get_model_attrs(cfg_model['model_name'])
print(f"Product-Daily Performance (n={len(attrs)}):")
attrs.head(10)

Product-Daily Performance (n=266):


,in_product_id,dt_date,customer_id_count,trans_id_count,quantity_sum,quantity_mean,quantity_median,price_total_sum,cost_total_sum,transaction_mean
0,prod1,20251001,2,2,4,2.00,2.0,6026.0,28720.0,3013.00
1,prod1,20251002,3,3,10,3.33,3.0,138302.0,93589.0,46100.67
2,prod1,20251003,4,6,20,3.33,3.0,241660.0,155758.0,40276.67
3,prod1,20251004,2,2,17,8.50,8.5,233720.0,165818.0,116860.00
4,prod1,20251005,3,4,25,6.25,2.5,377621.0,252954.0,94405.25
5,prod1,20251007,3,3,8,2.67,3.0,112181.0,81937.0,37393.67
6,prod1,20251008,3,4,12,3.00,3.0,159767.0,116551.0,39941.75
7,prod1,20251009,2,2,6,3.00,3.0,94879.0,57908.0,47439.50
8,prod1,20251010,3,3,11,3.67,3.0,130542.0,115228.0,43514.00
9,prod1,20251011,1,1,1,1.00,1.0,13544.0,10446.0,13544.00


In [9]:
# View summary statistics
print("Product-Daily Performance Summary:")
attrs[['quantity_sum', 'price_total_sum', 'transaction_mean']].describe()

Product-Daily Performance Summary:


,quantity_sum,price_total_sum,transaction_mean
count,266.000000,2.660000e+02,266.000000
mean,8.823308,2.011121e+05,82074.531015
std,7.762196,2.099235e+05,68983.283026
min,1.000000,2.145000e+03,2145.000000
25%,3.000000,5.896825e+04,34237.000000
50%,6.000000,1.383030e+05,67164.500000
75%,12.000000,2.655150e+05,106379.125000
max,43.000000,1.549612e+06,497296.000000


In [10]:
# View top performing product-day combinations
print("Top 10 Product-Day Combinations by Revenue:")
top_performers = attrs.nlargest(10, 'price_total_sum')[['in_product_id', 'dt_date', 'price_total_sum', 'quantity_sum', 'customer_id_count']]
top_performers

Top 10 Product-Day Combinations by Revenue:


,in_product_id,dt_date,price_total_sum,quantity_sum,customer_id_count
54,prod10,20251030,1549612.0,38,4
46,prod10,20251022,927842.0,26,3
229,prod8,20251020,899590.0,30,4
33,prod10,20251009,873364.0,23,5
227,prod8,20251017,833125.0,29,3
198,prod7,20251016,757506.0,21,3
35,prod10,20251011,714928.0,18,3
209,prod7,20251028,706107.0,22,3
233,prod8,20251024,705683.0,24,3
27,prod10,20251002,696925.0,18,6


## 8. Export to Excel

In [11]:
# Export model results to Excel
exporter = ExcelExporter(ctx)
output_file = f'outputs/{cfg_model["model_name"]}_export.xlsx'
exporter.export_model(cfg_model['model_name'], output_file, include_input=True)

print(f"✓ Export complete: {output_file}")
print("\nExcel tabs:")
print(f"  1. {cfg_model['input_dataset_name']} (input)")
print(f"  2. {cfg_model['model_name']}_filters")
print(f"  3. {cfg_model['model_name']}_attrs")

✓ Export complete: outputs/product_daily_export.xlsx

Excel tabs:
  1. transactions_filters (input)
  2. product_daily_filters
  3. product_daily_attrs


## 9. Save Context State

Save the complete context state for use in downstream notebooks:

In [12]:
# Save context state (datasets, config, metadata)
state_dir = save_context_state(ctx=ctx, base_cfg=base_cfg)

print(f"✓ Context state saved: {state_dir}")
print(f"  - Total datasets: {len(ctx.datasets)}")
print(f"\nTo load this state in another notebook:")
print(f"  from src.core.persistence import load_context_state")
print(f"  ctx, base_cfg = load_context_state('{state_dir}')")

✓ Context state saved: data\context_states\test_client_20251022_150907
  - Total datasets: 9

To load this state in another notebook:
  from src.core.persistence import load_context_state
  ctx, base_cfg = load_context_state('data\context_states\test_client_20251022_150907')
